## Importing required packages

In [2]:
import numpy as np
import cv2
from scipy.spatial import distance

In [ ]:
cd /content/drive/MyDrive/TASKS_ML_NETZWERK_ACADAMEY/DL/Social_distance_detection

/content/drive/MyDrive/TASKS_ML_NETZWERK_ACADAMEY/DL/Social_distance_detection


## Loading yolo weights,cfg and coco names

In [3]:
net = cv2.dnn.readNet('yolo/yolov3.weights','yolo/yolov3.cfg')
with open('yolo/coco.names','r')as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]
print(output_layers)
print(classes)

['yolo_82', 'yolo_94', 'yolo_106']
['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


## Used Yolo object detection to find only people and returns the confidence, coordinates and centroids of people 

In [4]:
def detect_people(frame, personIndex=0):
    """This function will detects the only people class and it finds the
    confidence,coordinates and centroids of people and returns those values"""
    results = []
    height, width = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(image=frame, scalefactor=1/255.0, size=(416, 416), mean=(0, 0, 0), swapRB=True,
                                 crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)
    boxes, centroids, confidences = [], [], []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if class_id == personIndex and confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                centroids.append((center_x, center_y))
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = (boxes[i][0], boxes[i][1], boxes[i][2], boxes[i][3])
            result = (confidences[i], (x, y, x + w, y + h), centroids[i])
            results.append(result)
    return results


In [4]:
ls

 Social_Distance_Detection.ipynb  'The CCTV People.mp4'   yolo/


## Here it finds the distance between the people, based on distance we will find who is violating social distance

In [7]:
def final(Frame):
    """This function finds the distance between the people,
    based on distance we will find who is violating social distance """
    frame1 = frame.copy()
    results = detect_people(frame1, personIndex=classes.index('person'))
    violate = set()
    if len(results) >= 2:
        centroids = np.array([r[2] for r in results])
        Distance = distance.cdist(centroids, centroids)
        for i in range(Distance.shape[0]):
            for j in range(i + 1, Distance.shape[1]):
                if Distance[i, j] < 85:
                    violate.add(i)
                    violate.add(j)

    for (i, (confi, coordinates, centroid)) in enumerate(results):
        (start_x, start_y, end_x, end_y) = coordinates
        color = (0, 255, 0)
        if i in violate:
            color = (0, 0, 255)
        cv2.rectangle(frame1, (start_x, start_y), (end_x, end_y), color, 2)
    text = f"social distance violations : {len(violate)}"
    # print(frame1)
    o = cv2.putText(frame1, text, (10, frame1.shape[0] - 25), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 0, 255), 3)
    return o


## Here we pass input (video file) it will finds the people who is violating and not violating the socal distance and also it saves the video file 

In [6]:
video = cv2.VideoCapture('The CCTV People.mp4')
frame_width = int(video.get(3))
frame_height = int(video.get(4))
size = (frame_width, frame_height)
result_video = cv2.VideoWriter('output2.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 10, size)
while True:
    ret, frame = video.read()
    if ret:
        out_img = final(frame)
        print(out_img)
        cv2.imshow('out',out_img)
        result_video.write(out_img)
        print('writing')
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
    else:
        break
video.release()
result_video.release()
cv2.destroyAllWindows()
print("Done processing video")


[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
writing
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
writing
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]


[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
writing
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
writing
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]


[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
writing
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
writing
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]


[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
writing
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
writing
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]


[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
writing
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
writing
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]


...

[[[ 66  19  22]
  [ 77  30  33]
  [ 74  43  44]
  ...
  [ 29  32  37]
  [ 28  31  36]
  [ 27  30  35]]

 [[ 73  26  29]
  [ 87  40  43]
  [170 139 140]
  ...
  [ 29  32  37]
  [ 28  31  36]
  [ 27  30  35]]

 [[119  81  83]
  [126  88  90]
  [169 148 147]
  ...
  [ 27  30  35]
  [ 27  30  35]
  [ 27  30  35]]

 ...

 [[ 93  88  89]
  [ 94  89  90]
  [ 94  89  90]
  ...
  [109 106 100]
  [109 106 100]
  [109 106 100]]

 [[ 78  70  72]
  [ 92  84  86]
  [ 95  90  91]
  ...
  [111 105 100]
  [113 105 100]
  [113 105 100]]

 [[ 82  74  76]
  [ 92  84  86]
  [ 78  73  74]
  ...
  [111 105 100]
  [113 105 100]
  [113 105 100]]]
writing
[[[ 66  19  22]
  [ 77  30  33]
  [ 74  43  44]
  ...
  [ 29  32  37]
  [ 28  31  36]
  [ 27  30  35]]

 [[ 73  26  29]
  [ 87  40  43]
  [170 139 140]
  ...
  [ 29  32  37]
  [ 28  31  36]
  [ 27  30  35]]

 [[119  81  83]
  [126  88  90]
  [169 148 147]
  ...
  [ 27  30  35]
  [ 27  30  35]
  [ 27  30  35]]

 ...

 [[ 95  89  92]
  [ 97  91  94]
  [ 97  91  

[[[ 66  19  22]
  [ 77  30  33]
  [ 74  43  44]
  ...
  [ 29  32  37]
  [ 28  31  36]
  [ 27  30  35]]

 [[ 73  26  29]
  [ 87  40  43]
  [170 139 140]
  ...
  [ 29  32  37]
  [ 28  31  36]
  [ 27  30  35]]

 [[119  81  83]
  [126  88  90]
  [169 148 147]
  ...
  [ 27  30  35]
  [ 27  30  35]
  [ 27  30  35]]

 ...

 [[ 99  94  93]
  [ 99  94  93]
  [ 99  94  93]
  ...
  [106 104 100]
  [106 104 100]
  [106 104 100]]

 [[ 84  76  76]
  [ 99  91  91]
  [101  96  95]
  ...
  [107 105 101]
  [107 105 101]
  [107 105 101]]

 [[ 88  80  80]
  [ 97  89  89]
  [ 84  79  78]
  ...
  [109 107 103]
  [109 107 103]
  [109 107 103]]]
writing
[[[ 66  19  22]
  [ 77  30  33]
  [ 74  43  44]
  ...
  [ 29  32  37]
  [ 28  31  36]
  [ 27  30  35]]

 [[ 73  26  29]
  [ 87  40  43]
  [170 139 140]
  ...
  [ 29  32  37]
  [ 28  31  36]
  [ 27  30  35]]

 [[119  81  83]
  [126  88  90]
  [169 148 147]
  ...
  [ 27  30  35]
  [ 27  30  35]
  [ 27  30  35]]

 ...

 [[ 99  94  93]
  [ 99  94  93]
  [ 99  94  

[[[ 66  19  22]
  [ 77  30  33]
  [ 74  43  44]
  ...
  [ 29  32  37]
  [ 28  31  36]
  [ 27  30  35]]

 [[ 73  26  29]
  [ 87  40  43]
  [170 139 140]
  ...
  [ 28  31  36]
  [ 28  31  36]
  [ 27  30  35]]

 [[119  81  83]
  [126  88  90]
  [169 148 147]
  ...
  [ 28  31  36]
  [ 27  30  35]
  [ 27  30  35]]

 ...

 [[102  97  96]
  [101  96  95]
  [102  97  96]
  ...
  [111 105 100]
  [111 105 100]
  [111 105 100]]

 [[ 87  79  79]
  [102  94  94]
  [105 100  99]
  ...
  [111 105 100]
  [111 105 100]
  [111 105 100]]

 [[ 94  86  86]
  [102  94  94]
  [ 87  82  81]
  ...
  [111 105 100]
  [111 105 100]
  [111 105 100]]]
writing
[[[ 66  19  22]
  [ 77  30  33]
  [ 74  43  44]
  ...
  [ 29  32  37]
  [ 28  31  36]
  [ 27  30  35]]

 [[ 73  26  29]
  [ 87  40  43]
  [170 139 140]
  ...
  [ 28  31  36]
  [ 28  31  36]
  [ 27  30  35]]

 [[119  81  83]
  [126  88  90]
  [169 148 147]
  ...
  [ 28  31  36]
  [ 27  30  35]
  [ 27  30  35]]

 ...

 [[102  97  96]
  [101  96  95]
  [102  97  

[[[ 73  26  29]
  [ 84  37  40]
  [ 81  50  51]
  ...
  [ 29  32  37]
  [ 28  31  36]
  [ 27  30  35]]

 [[ 79  32  35]
  [ 93  46  49]
  [176 145 146]
  ...
  [ 28  31  36]
  [ 28  31  36]
  [ 27  30  35]]

 [[120  82  84]
  [127  89  91]
  [170 149 148]
  ...
  [ 28  31  36]
  [ 27  30  35]
  [ 27  30  35]]

 ...

 [[100 100  98]
  [ 98  98  96]
  [ 98  98  96]
  ...
  [111 105 100]
  [111 105 100]
  [111 105 100]]

 [[ 88  85  84]
  [103 100  99]
  [105 105 103]
  ...
  [111 105 100]
  [111 105 100]
  [111 105 100]]

 [[ 97  94  93]
  [104 101 100]
  [ 90  90  88]
  ...
  [111 105 100]
  [111 105 100]
  [111 105 100]]]
writing
[[[ 73  26  29]
  [ 84  37  40]
  [ 81  50  51]
  ...
  [ 29  32  37]
  [ 28  31  36]
  [ 27  30  35]]

 [[ 79  32  35]
  [ 93  46  49]
  [176 145 146]
  ...
  [ 28  31  36]
  [ 28  31  36]
  [ 27  30  35]]

 [[120  82  84]
  [127  89  91]
  [170 149 148]
  ...
  [ 28  31  36]
  [ 27  30  35]
  [ 27  30  35]]

 ...

 [[100 100  98]
  [ 98  98  96]
  [ 98  98  

[[[ 73  26  29]
  [ 84  37  40]
  [ 81  50  51]
  ...
  [ 29  32  37]
  [ 29  32  37]
  [ 29  32  37]]

 [[ 79  32  35]
  [ 93  46  49]
  [176 145 146]
  ...
  [ 29  32  37]
  [ 29  32  37]
  [ 29  32  37]]

 [[120  82  84]
  [127  89  91]
  [170 149 148]
  ...
  [ 29  32  37]
  [ 29  32  37]
  [ 29  32  37]]

 ...

 [[100 100  98]
  [ 98  98  96]
  [100 100  98]
  ...
  [111 105 100]
  [111 105 100]
  [111 105 100]]

 [[ 88  85  84]
  [102  99  98]
  [105 105 103]
  ...
  [111 105 100]
  [111 105 100]
  [111 105 100]]

 [[ 97  94  93]
  [103 100  99]
  [ 90  90  88]
  ...
  [111 105 100]
  [111 105 100]
  [111 105 100]]]
writing
[[[ 73  26  29]
  [ 84  37  40]
  [ 81  50  51]
  ...
  [ 29  32  37]
  [ 29  32  37]
  [ 29  32  37]]

 [[ 79  32  35]
  [ 93  46  49]
  [176 145 146]
  ...
  [ 29  32  37]
  [ 29  32  37]
  [ 29  32  37]]

 [[120  82  84]
  [127  89  91]
  [170 149 148]
  ...
  [ 29  32  37]
  [ 29  32  37]
  [ 29  32  37]]

 ...

 [[100 100  98]
  [ 98  98  96]
  [100 100  